In [1]:
import torch
from torchvision import datasets, transforms
from collections import OrderedDict
from multiae import mcVAE, DVCCA
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline 

c:\Users\anala\Anaconda3\envs\testing_refactor\lib\site-packages\tqdm-4.64.0-py3.8.egg\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the MNIST data

In [2]:
MNIST_1 = datasets.MNIST('./data/MNIST', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
    ]))



data_1 = MNIST_1.train_data[:, :, :14].reshape(-1,392).float()/255.
data_2 = MNIST_1.train_data[:, :, 14:].reshape(-1,392).float()/255.
target = MNIST_1.train_labels

MNIST_1 = datasets.MNIST('./data/MNIST', train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor()
    ]))

data_test_1 = MNIST_1.test_data[:, :, :14].reshape(-1,392).float()/255.
data_test_2 = MNIST_1.test_data[:, :, 14:].reshape(-1,392).float()/255.
target_test = MNIST_1.test_labels.numpy()


c:\Users\anala\Anaconda3\envs\testing_refactor\lib\site-packages\torchvision-0.13.1-py3.8-win-amd64.egg\torchvision\datasets\mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
c:\Users\anala\Anaconda3\envs\testing_refactor\lib\site-packages\torchvision-0.13.1-py3.8-win-amd64.egg\torchvision\datasets\mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
c:\Users\anala\Anaconda3\envs\testing_refactor\lib\site-packages\torchvision-0.13.1-py3.8-win-amd64.egg\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\anala\Anaconda3\envs\testing_refactor\lib\site-packages\torchvision-0.13.1-py3.8-win-amd64.egg\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### Define parameters

In [3]:
input_dim=[392,392]
max_epochs = 10
batch_size = 2000
latent_dim = 2

### Define models

In [4]:
mcvae = mcVAE(
        cfg="./config/example_mnist.yaml",
        input_dim=input_dim,
        z_dim=latent_dim,
    )
dvcca = DVCCA(
        cfg="./config/example_mnist.yaml",
        input_dim=input_dim,
        z_dim=latent_dim,
    )


model_name:
  mcVAE
model:
  use_GPU: false
  save_model: true
  seed_everything: true
  seed: 42
  z_dim: 64
  learning_rate: 0.001
  sparse: false
  threshold: 0
  beta: 1
  
datamodule:
  _target_: multiae.base.dataloaders.MultiviewDataModule
  batch_size: null
  is_validate: true
  train_size: 0.9
  
encoder:
  _target_: multiae.models.layers.VariationalEncoder
  hidden_layer_dim:
  - 512
  - 512
  bias: true
  non_linear: true
  enc_dist:
    _target_: multiae.base.distributions.Normal
  
decoder:
  _target_: multiae.architectures.mlp.Decoder
  hidden_layer_dim:
  - 512
  - 512
  bias: true
  non_linear: true
  dec_dist:
    _target_: multiae.base.distributions.Default
  init_logvar: -3
  
prior:
  _target_: multiae.base.distributions.Normal
  loc: 0
  scale: 1
  
trainer:
  _target_: pytorch_lightning.Trainer
  gpus: 0
  max_epochs: 10
  deterministic: false
  log_every_n_steps: 2
  resume_from_checkpoint: null
  
callbacks:
  model_checkpoint:
    _target_: pytorch_lightning.cal

NameError: name 'exit' is not defined

### Train the models

In [ ]:

mcvae.fit(data_1, data_2, max_epochs=max_epochs, batch_size=batch_size)
dvcca.fit(data_1, data_2, max_epochs=max_epochs, batch_size=batch_size)

### Create latent plots - how well does the latent space separate MNIST labels?

In [ ]:
mcvae_latent = mcvae.predict_latents(data_test_1, data_test_2)
dvcca_latent = dvcca.predict_latents(data_test_1, data_test_2)

In [ ]:
colors =        {0: 'tab:blue',
                 1:'tab:orange',
                 2: 'r',
                 3: 'c',
                 4: 'm',
                 5: 'y',
                 6: 'g',
                 7: 'k',
                 8: 'tab:pink',
                 9: 'tab:gray'
                 }
fig=plt.figure(figsize=(8,6)) 
ax1 = fig.add_subplot(1, 1, 1)
ax1.scatter(dvcca_latent[0][:,0], dvcca_latent[0][:,1], c=pd.Series(target_test).map(colors))
ax1.set_title('DVCCA latent vectors')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.tight_layout()
plt.show()

In [ ]:
fig=plt.figure(figsize=(8,6)) 
ax1 = fig.add_subplot(1, 2, 1)
ax1.scatter(mcvae_latent[0][:,0], mcvae_latent[0][:,1], c=pd.Series(target_test).map(colors))
ax1.set_title('mcVAE latent vectors view 1')
ax2 = fig.add_subplot(1, 2, 2)
ax2.scatter(mcvae_latent[1][:,0], mcvae_latent[1][:,1], c=pd.Series(target_test).map(colors))
ax2.set_title('mcVAE latent vectors view 2')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.tight_layout()
plt.show()

### mcVAE reconstruction example

In [ ]:
pred = mcvae.predict_reconstruction(data_1, data_2)

### Reconstruction plots - how well can the VAE do same view reconstruction?

In [ ]:
data_sample = data_1[20]
#indices: view 1 latent, view 1 decoder, sample 21
pred_sample = pred[0][0][20]

In [ ]:
fig, axarr = plt.subplots(1, 2)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
axarr[0].imshow(data_sample.reshape(28,28))
axarr[1].imshow(pred_sample.reshape(28,28))

### Reconstruction plots - how well can the VAE do cross view reconstruction?

In [ ]:
data_sample = data_1[20]
#indices: view 1 latent, view 2 decoder, sample 21
pred_sample = pred[1][1][20]

In [ ]:
fig, axarr = plt.subplots(1, 2)
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
axarr[0].imshow(data_sample.reshape(28,28))
axarr[1].imshow(pred_sample.reshape(28,28))